# Building Conversational Chatbot Using Trained Models

### Step 1: Import Required Libraries

In [ ]:
# install sentence_transformers if not already installed
%pip install sentence_transformers

In [ ]:
import os
import joblib
import numpy as np
import pandas as pd
import tkinter as tk
from tkinter import messagebox
import re
from pathlib import Path
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import json
from functools import lru_cache
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns

import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(project_root)
import model.language_manager.lang_spelling as LanguageSpellingChecker

### Step 2: Load Models and Data

##### 2.1 Set up paths

In [ ]:
script_dir = os.path.abspath(os.path.join(os.getcwd(), "../.."))
models_path = os.path.join(script_dir, "model/training_manager/models.pkl")

##### 2.2 Load models and data

In [ ]:
trained_models = joblib.load(models_path)
print(f"✅ Loaded {len(trained_models)} trained models")

##### 2.3 Check for loaded model details

In [ ]:
model_infos = []
for result in trained_models:
    model_infos.append({
        "Model": result.get('model', 'No model available').__class__.__name__,
        "Predictor": result.get('predictor', 'No predictor available'),
        "Features": result.get('features', 'No features available'),
        "Description": result.get('description', 'No description available')
    })
models_df = pd.DataFrame(model_infos)
with pd.option_context('display.max_colwidth', None):
    display(models_df)

##### 2.4 Initialize embedder

In [ ]:
embedder = SentenceTransformer('paraphrase-MiniLM-L6-v2')
print("✅ Loaded sentence embedding model")

### Step 3: Model Selection Logic

In [ ]:
import pandas as pd

def select_model(user_input):
    user_input_lower = user_input.lower().strip()
    user_input_lower = LanguageSpellingChecker.correct_spelling(user_input_lower)
    print(f"\n🔍 Analyzing input: '{user_input_lower}'")
    
    # --------------------------------------------------
    # 1. First try exact keyword matching
    # --------------------------------------------------
    for model in trained_models:
        # Match with predictor name (exact)
        predictor = model.get('predictor', '').lower()
        if predictor and predictor in user_input_lower:
            print(f"✅ Exact predictor match: {predictor}")
            print(f"⭐ Selected Model: {model.get('description', 'No description available')}")
            return model

        # Cosine similarity on predictor
        if embedder and predictor:
            sim = cosine_similarity(
                embedder.encode([user_input_lower]),
                embedder.encode([predictor])
            )[0][0]
            if sim > 0.2:
                print(f"✅ Predictor semantic match: '{predictor}' (similarity: {sim:.2f})")
                print(f"⭐ Selected Model: {model.get('description', 'No description available')}")
                return model
            
        # Match with features (exact)
        features = model.get('features', [])
        matched_features = [feat for feat in features if feat.lower() in user_input_lower]
        if matched_features:
            print(f"✅ Feature match: {matched_features}")
            print(f"⭐ Selected Model: {model.get('description', 'No description available')}")
            return model

        # Cosine similarity on features
        if embedder and features:
            feature_sims = [
                cosine_similarity(
                    embedder.encode([user_input_lower]),
                    embedder.encode([feat.lower()])
                )[0][0] for feat in features
            ]
            max_sim = max(feature_sims) if feature_sims else 0
            if max_sim > 0.2:
                best_feat = features[feature_sims.index(max_sim)]
                print(f"✅ Feature semantic match: '{best_feat}' (similarity: {max_sim:.2f})")
                print(f"⭐ Selected Model: {model.get('description', 'No description available')}")
                return model
    
    # --------------------------------------------------
    # 2. Special handling for tag classification models
    # --------------------------------------------------
    tag_models = [m for m in trained_models 
                if m.get('vectorizer') and m.get('predictor', '').lower() == 'tag']
    
    if tag_models and embedder:
        tag_model = tag_models[0]  # Assuming one tag model
        label_encoder = tag_model.get('label_encoder')
        
        if label_encoder:
            # Get all possible tags
            tag_classes = label_encoder.classes_
            print("🏷 Available tags (vertical):")
            for tag in tag_classes:
                print(f" - {tag}")
            
            # Calculate similarity between input and each tag
            tag_embeddings = embedder.encode(tag_classes)
            user_embedding = embedder.encode([user_input_lower])
            sims = cosine_similarity(user_embedding, tag_embeddings)[0]
            
            # Get best match
            best_idx = np.argmax(sims)
            best_tag = tag_classes[best_idx]
            similarity = sims[best_idx]
            
            # Display semantic tag matching results in a DataFrame
            tag_scores_df = pd.DataFrame({
                'Tag': tag_classes,
                'Similarity': sims
            }).sort_values('Similarity', ascending=False).reset_index(drop=True)
            display(tag_scores_df)
            
            # If we have a good match, return it directly
            if similarity > 0.2:
                print(f"🏷 Best semantic tag match: {best_tag} (similarity: {similarity:.2f})")
                print(f"⭐ Selected Model: {tag_model.get('description', 'No description available')}")
                # Return special structure for tag matches
                return {
                    'model_type': 'semantic_tag_match',
                    'matched_tag': best_tag,
                    'similarity': similarity,
                    'original_model': tag_model  # Keep reference to original model
                }
            else:
                print(f"⚠ No strong tag match (best: {best_tag} @ {similarity:.2f})")
    
    # --------------------------------------------------
    # 3. Fallback to general model description matching
    # --------------------------------------------------
    if embedder:
        print("🔎 Trying general model matching...")
        try:
            model_descriptions = []
            for idx, model_info in enumerate(trained_models):
                predictor = model_info.get('predictor', 'Unknown')
                features = ', '.join(model_info.get('features', []))
                desc = f"Predicts {predictor} using {features}"
                model_descriptions.append(desc)
            
            description_embeddings = embedder.encode(model_descriptions)
            user_embedding = embedder.encode([user_input_lower])
            sims = cosine_similarity(user_embedding, description_embeddings)[0]
            
            # Get top 3 matches
            top_indices = np.argsort(sims)[-3:][::-1]
            
            print("Top model matches:")
            for idx in top_indices:
                print(f" - #{idx}: {model_descriptions[idx]} (score: {sims[idx]:.2f})")
            
            best_idx = top_indices[0]
            if sims[best_idx] > 0.2:
                print(f"✅ Selected model #{best_idx} (score: {sims[best_idx]:.2f})")
                return trained_models[best_idx]
            
        except Exception as e:
            print(f"⚠ Model matching error: {e}")
    
    # --------------------------------------------------
    # 4. Final fallback
    # --------------------------------------------------
    print("❌ No suitable model found")
    return None

### Step 4: Enhanced Input Preparation

In [ ]:
def prepare_input(model_info, user_input):
    """Prepare input data based on model type"""
    if model_info.get('vectorizer'):
        # Text classification model
        return model_info['vectorizer'].transform([user_input])
    else:
        # Structured data model
        features = {}
        input_text = user_input.lower()
        
        # Handle one-hot encoded features
        expected_columns = model_info.get('X_train_columns', [])
        for col in expected_columns:
            if '_' in col:  # One-hot encoded feature
                feature, value = col.split('_', 1)
                if feature.lower() in input_text and value.lower() in input_text:
                    features[col] = 1
                else:
                    features[col] = 0
            else:  # Regular feature
                features[col] = 1 if col.lower() in input_text else 0
        
        # Display expected columns as a DataFrame (vertical format)
        display(pd.DataFrame(expected_columns, columns=["Expected Feature"]))
        # Convert to DataFrame with correct column names
        return pd.DataFrame([features], columns=expected_columns)

### Step 5: Response Generation

In [ ]:
def generate_response(model_info, user_input):
    if not model_info:
        return "⚠️ No model found for this query"
    
    # Handle direct tag matches
    if 'matched_tag' in model_info:
        return f"🏷️ Based on your query, the most relevant tag is: {model_info['matched_tag']} " \
               f"(match confidence: {model_info['similarity']:.0%})"
    
    # Original model prediction logic
    try:
        model = model_info.get("model")
        if model is None:
            return "⚠️ No model available"
            
        processed_input = prepare_input(model_info, user_input)
        prediction = model.predict(processed_input)[0]
        
        if 'label_encoder' in model_info:
            try:
                prediction = model_info['label_encoder'].inverse_transform([prediction])[0]
                print(f"📌 Model predicted tag: {prediction}")
            except ValueError as e:
                print(f"⚠️ Label encoding error: {e}")
                return "⚠️ Could not interpret the response"
        
        confidence = 0.8
        if hasattr(model, "predict_proba"):
            proba = model.predict_proba(processed_input)
            confidence = np.max(proba)
            print(f"🔢 Confidence scores: {np.round(proba, 2)}")
        
        return f"✈️ The predicted {model_info.get('predictor', 'attribute')} is: {prediction} " \
               f"(confidence: {confidence:.0%})"
        
    except Exception as e:
        print(f"⚠️ Error: {e}")
        return "⚠️ Sorry, I encountered an error"

### Step 6: Chatbot GUI Interface

In [ ]:
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation/symbols
    text = re.sub(r'\s+', ' ', text)     # Remove extra spaces
    return text.strip().lower()

In [ ]:
def ask_question_gui():
    def on_submit():
        user_input = entry.get()        
        cleaned_input = clean_text(user_input)
        if not cleaned_input:
            messagebox.showwarning("Missing Input", "Please ask a question.")
            return
        
        # Try model prediction
        model_info = select_model(cleaned_input)
        if model_info:
            response = generate_response(model_info, cleaned_input)
            if response:
                messagebox.showinfo("TrippoBot Response", response)
                return
        
        # Fallback response
        messagebox.showinfo("TrippoBot Response", 
                          "I couldn't find that information in my travel database. Could you try asking differently?")

    window = tk.Tk()
    window.title("TrippoBot - Ask a Travel Question")

    tk.Label(window, text="Enter your travel question:").pack(pady=10)
    entry = tk.Entry(window, width=80)
    entry.pack(padx=30, pady=15)

    submit_btn = tk.Button(window, text="Ask TrippoBot", command=on_submit)
    submit_btn.pack(pady=10)

    window.mainloop()

In [ ]:
ask_question_gui()

### Step 7: Extract metrics from trained models into a structured format

In [ ]:
def extract_model_metrics(trained_models):
    """Extract metrics from trained models into a structured format"""
    metrics_list = []
    
    for model_info in trained_models:
        metrics = model_info['metrics']
        model_data = {
            'Model': model_info['model'].__class__.__name__,
            'Task': f"Predict {model_info['predictor']} using {', '.join(model_info['features'])}",
            'Accuracy': metrics.get('accuracy', np.nan),
            'F1_Score': metrics.get('f1', np.nan),
            'Precision': metrics.get('precision', np.nan),
            'Recall': metrics.get('recall', np.nan),
            'ROC_AUC': metrics.get('roc_auc', np.nan),
            'Avg_Confidence': np.mean(metrics.get('confidence', [np.nan])) * 100 if metrics.get('confidence') is not None else np.nan
        }
        metrics_list.append(model_data)
    
    return pd.DataFrame(metrics_list)

# Load your trained models
try:
    trained_models = joblib.load(models_path)
    metrics_df = extract_model_metrics(trained_models)
    
    print("\nMODEL PERFORMANCE METRICS")
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', None, 'display.max_colwidth', None):
        display(metrics_df)
    
except FileNotFoundError:
    print(f"Error: File not found at {models_path}")
except Exception as e:
    print(f"Error: {str(e)}")

### Step 8: Create Dynamic Visualizations
##### Comparison Bar Chart

In [ ]:
def plot_model_comparison(metrics_df):
    """Create a dynamic comparison plot based on available metrics"""
    # Select only numeric columns for comparison
    numeric_cols = metrics_df.select_dtypes(include=[np.number]).columns.tolist()
    
    # Set up plot
    fig, axes = plt.subplots(nrows=len(numeric_cols), figsize=(10, 5*len(numeric_cols)), squeeze=False)
    fig.suptitle('Model Performance Comparison', y=1.02)
    
    for idx, metric in enumerate(numeric_cols):
        ax = axes[idx][0]
        x = np.arange(len(metrics_df))
        width = 0.35
        
        # Get values and convert to percentages where appropriate
        values = metrics_df[metric].values
        if metric != 'Avg_Confidence':  # Most metrics are already in 0-1 scale
            values = [v * 100 for v in values]
        
        bars = ax.bar(x, values, width)
        ax.set_title(metric.replace('_', ' '))
        ax.set_ylabel('Score (%)')
        ax.set_xticks(x)
        ax.set_xticklabels(metrics_df['Model'])
        ax.set_ylim(0, 105)
        
        # Add value labels
        for bar in bars:
            height = bar.get_height()
            ax.annotate(f'{height:.1f}',
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),
                        textcoords="offset points",
                        ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()

plot_model_comparison(metrics_df)

##### Radar Chart for Comprehensive Comparison

In [ ]:
def plot_radar_comparison(metrics_df):
    """Create a radar chart comparing all models"""
    # Select and normalize metrics
    plot_metrics = ['Accuracy', 'F1_Score', 'Precision', 'Recall', 'ROC_AUC', 'Avg_Confidence']
    plot_df = metrics_df.set_index('Model')[plot_metrics]
    
    # Normalize to 0-100 scale
    plot_df = plot_df.apply(lambda x: x*100 if x.name != 'Avg_Confidence' else x)
    
    # Number of variables we're plotting
    categories = list(plot_df.columns)
    N = len(categories)
    
    # Calculate angle for each axis
    angles = [n / float(N) * 2 * np.pi for n in range(N)]
    angles += angles[:1]  # Complete the loop
    
    # Initialize the radar plot
    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))
    
    # Draw one axe per variable and add labels
    plt.xticks(angles[:-1], categories)
    
    # Draw ylabels
    ax.set_rlabel_position(0)
    plt.yticks([20, 40, 60, 80, 100], ["20", "40", "60", "80", "100"], color="grey", size=7)
    plt.ylim(0, 100)
    
    # Plot each model
    for idx, row in plot_df.iterrows():
        values = row.values.flatten().tolist()
        values += values[:1]  # Complete the loop
        ax.plot(angles, values, linewidth=2, linestyle='solid', label=idx)
        ax.fill(angles, values, alpha=0.1)
    
    # Add legend and title
    plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
    plt.title('Model Performance Radar Chart', size=14, y=1.1)
    
    plt.tight_layout()
    plt.show()

plot_radar_comparison(metrics_df)

### - END